### Loading Function

In [0]:
from pyspark.sql.functions import *
import pyspark.sql.functions as F

In [0]:
container="veersacontainer"
storage_account="storage12092004"

### creating schema

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS veersadatabricks.silver_final
MANAGED LOCATION 'abfss://veersacontainer@storage12092004.dfs.core.windows.net/silver/';

### Case Table
- The table contains data related to infection cases, including details about the location and nature of each case. 
- checking for null values of caseid
- caseId is id of infection case
- confirmed column has been renamed to confirmedCases
- table contains 8 fields - including latitude and longitude fields to preserve as much clean data as possible

In [0]:
case_silver = spark.table("veersadatabricks.silver.case_table_silver")
#case_bronze.printSchema()
case_table_silver = (
    case_silver
    .withColumnRenamed("_case_id", "caseId")
    .withColumnRenamed("confirmed", "confirmedCases")
    .withColumn("confirmedCases", col("confirmedCases").cast("int"))
    .withColumn("city", trim(col("city")))
    .filter(
        col("caseId").isNotNull() &
        col("confirmedCases").isNotNull() 
        #&
        #(col("city").isNotNull() & (col("city") != "-"))
    )
    .dropDuplicates()
)

In [0]:
case_table_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .option("path", f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/CaseTable")\
    .saveAsTable("veersadatabricks.silver_final.case_table_silver")

### PatientInfo Table
- The table contains data related to patient infections, including demographic information and infection details. It can be used to analyze trends in infection cases, track patient outcomes, and understand the spread of infections across different regions.
- converted contact_number (number of people patient came in contact with) to integer
- cleaned age, sex, and symptom_onset_column by converting them to appropriate types
- trimmed and capitalized each word for country, province and city + removed whitespaces
- applied filters like patient_id and city should not be null
- dropped rows where country is not korea

In [0]:
%sql
DROP TABLE IF EXISTS veersadatabricks.silver_final.patientinfo_silver;


In [0]:
patientInfo_df = spark.table("veersadatabricks.Silver.patientinfo_silver")
#patientInfo_df.printSchema()
patientInfo_silver = (
    patientInfo_df
    # ---- Age cleaning ----
    .filter(lower(trim(col("is_current"))) == "true")
    .withColumn("age", 
                F.when(F.col("age").rlike("^[0-9]+$"), F.col("age").cast("int"))
                .when(F.col("age").rlike("^[0-9]+s$"), F.regexp_extract("age", "([0-9]+)", 1).cast("int"))
                 .otherwise(None))   # invalid ages set to null
                 
    # ---- Contact number cleaning ----
    .withColumn("contact_number", 
                F.when(F.col("contact_number").rlike("^[0-9]+$"), 
                       F.col("contact_number").cast("int"))
                 .otherwise(None))   # non-numeric → null
                 
    # ---- Standardize sex ----
    .withColumn("sex", 
                F.lower(F.trim(F.col("sex"))))  # "Male " → "male"
    
    # ---- Dates cleaning ----
    .withColumn("symptom_onset_date", 
                F.to_date("symptom_onset_date", "yyyy-MM-dd"))
    
    .withColumn("country", F.initcap(F.trim(F.col("country"))))
    .withColumn("province", F.initcap(F.trim(F.col("province"))))
    .withColumn("city", F.initcap(F.trim(F.col("city"))))

    # ---- infected_by cleaning ----
    .withColumn("infected_by", 
        F.when(F.col("infected_by").rlike("^[0-9]+$"), F.col("infected_by").cast("long"))
         .otherwise(None))
    
    # ---- Infection case standardization ----
    .withColumn("infection_case", F.initcap(F.trim(F.col("infection_case"))))
    .filter(
        col("patient_id").isNotNull() &
        col("city").isNotNull() &
        (col("city") != "-")
    )
)
patientInfo_silver = patientInfo_silver.withColumn(
    "patient_id", F.col("patient_id").cast("long")
)
patientInfo_silver = patientInfo_silver.filter(lower(trim(col("country"))) == "korea")
#patientInfo_silver = patientInfo_silver.filter(lower(trim(col("is_current"))) == "true")

In [0]:
patientInfo_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .option("path", f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/PatientInfo") \
    .saveAsTable("veersadatabricks.silver_final.patientinfo_silver")

### Region Table
- dropped education related fields as the spread of covid is not specifically related to them
- applied filters for not null city
- filling missing values where needed

In [0]:
region_df = spark.table("veersadatabricks.silver.region_silver")
region_silver_df = region_df.select(
    col("code").cast("long"),
    col("province").cast("string"),
    col("city").cast("string"),
    col("latitude").cast("double"),
    col("longitude").cast("double"),
    col("elderly_population_ratio").cast("double"),
    col("elderly_alone_ratio").cast("double"),
    col("nursing_home_count").cast("long")
)

# Silver layer transformations
region_silver_df = (
    region_silver_df
    # Fill missing values where appropriate
    .fillna({
        "nursing_home_count": 0,
        "elderly_population_ratio": 0.0,
        "elderly_alone_ratio": 0.0
    })

    # Deduplicate based on region code
    .dropDuplicates(["code"])
    .filter(
        col("city").isNotNull() &
        (col("city") != "-")
    )
)


In [0]:
region_silver_df.write.format("delta") \
    .mode("overwrite") \
    .option("path", f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/Region") \
    .saveAsTable("veersadatabricks.silver_final.region_silver")

### Time table
- dropped time row

In [0]:
time_df = spark.table("veersadatabricks.silver.time_silver")

In [0]:
silver_time_df = (
    time_df
    .drop("time")  # dropping redundant time column
    .withColumn("date", F.to_date("date", "yyyy-MM-dd"))  # ensuring proper date type
)

In [0]:
silver_time_df.write.format("delta") \
    .mode("overwrite") \
    .option("path", f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/Time") \
    .saveAsTable("veersadatabricks.silver_final.time_silver")

### Time Age
- dropped time column as it only contained 0 or midnight as value
- converted age groups into their lower limit i.e. 10s to 10, can be converted to 10-19 during visualization for better readability

In [0]:
time_age_df = spark.table("veersadatabricks.silver.timeage_silver")

In [0]:
silver_time_age_df = ( 
    time_age_df 
    .drop("time") # drop redundant midnight column 
    .withColumn("date", F.to_date("date", "yyyy-MM-dd")) 
    .withColumn("age_group", F.regexp_replace("age", "s", "").cast("int"))
    #convert "20s" → 20 .drop("age"), regexp_replace, removes the s from age column, converts the remaining number to int using cast function, droppping age column can be done, keeping it for now to remember better
)

In [0]:
silver_time_age_df.write.format("delta") \
    .mode("overwrite") \
    .option("path", f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/TimeAge") \
    .saveAsTable("veersadatabricks.silver_final.timeage_silver")

### Time Province

In [0]:
time_province_df = spark.table("veersadatabricks.silver.timeprovince_silver")

In [0]:
silver_time_province_df = (
    time_province_df
    # ensure proper date type
    .withColumn("date", F.to_date("date", "yyyy-MM-dd"))
    # drop the redundant time column
    .drop("time")
)

In [0]:
silver_time_province_df.write.format("delta") \
    .mode("overwrite") \
    .option("path", f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/TimeProvince") \
    .saveAsTable("veersadatabricks.silver_final.timeprovince_silver")

### Time Gender

In [0]:
time_gender_df = spark.table("veersadatabricks.silver.timegender_silver")

In [0]:
silver_time_gender_df = (
    time_gender_df
    .drop("time")
    .withColumn("date", F.to_date("date", "yyyy-MM-dd"))   # standardize date type
)

In [0]:
silver_time_gender_df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema","true")\
    .option("path", f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/TimeGender") \
    .saveAsTable("veersadatabricks.silver_final.timegender_silver")

### Search Trend
- ensuring date is not null
- dropping duplicates
- filling 0.0 for keyword searches

In [0]:
search_trend_df = spark.table("veersadatabricks.silver.searchtrend_silver")

In [0]:
# Transformations
search_trend_silver = (
    search_trend_df
    .dropDuplicates(["date"])  # keep unique record per date
    .filter(F.col("date").isNotNull())  # remove null dates
    .fillna(0.0, subset=["cold", "flu", "pneumonia", "coronavirus"])  # fill missing search volumes
)

In [0]:
search_trend_silver.write.format("delta") \
    .mode("overwrite") \
    .option("path", f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/SearchTrned") \
    .saveAsTable("veersadatabricks.silver_final.searchtrend_silver")

### Weather
- dropping duplicates
- rounding column values to 2 decimal places

In [0]:
weather_df = spark.table("veersadatabricks.silver.weather_silver")

# Transformations
weather_silver = (
    weather_df
    .dropDuplicates(["province", "date"])   # keep unique records per province & date
    .withColumn("province", F.initcap("province"))  # standardize casing
    .filter(F.col("date").isNotNull())       # remove rows without date
    .withColumn("avg_temp", F.round("avg_temp", 2))
    .withColumn("min_temp", F.round("min_temp", 2))
    .withColumn("max_temp", F.round("max_temp", 2))
    .withColumn("precipitation", F.coalesce("precipitation", F.lit(0.0)))
    .withColumn("avg_relative_humidity", F.round("avg_relative_humidity", 2))
)

In [0]:
weather_silver.write.format("delta") \
    .mode("overwrite") \
    .option("path", f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/Weather") \
    .saveAsTable("veersadatabricks.silver_final.weather_silver")